In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, precision_score, confusion_matrix, f1_score, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import optuna
import joblib

In [40]:
df = pd.read_csv("consumo_casa.csv")


df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Dia'] = df['Fecha'].dt.day
df['Hora'] = df['Fecha'].dt.hour
df['Minuto'] = df['Fecha'].dt.minute

del df['Unnamed: 0']
del df['Fecha']

columnas = df.columns.tolist()
ultimas_tres_columnas = columnas[-3:]
nuevas_columnas = ultimas_tres_columnas + columnas[:-3]
df = df[nuevas_columnas]

df.columns = df.columns.str.replace(' ', '')
df = df.rename(columns={'Medidor[W]': 'Medidor'})
df1 = df.iloc[:, :4] # Selecciona las dos primeras columnas
df2 = df.iloc[:, 4:] # Selecciona las columnas desde la tercera en adelante
df2 = df2.applymap(lambda x: 1 if x > 0 else x)

C:\Users\Asus\AppData\Local\Temp\ipykernel_10616\2218950289.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(lambda x: 1 if x > 0 else x)


In [18]:
def modelosPorDispositivo(index):
    X = df1
    y = df2.iloc[:, index]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
    #X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=0) # 0.25 * 0.8 = 0.2

    model_preds = []
    def fit_model(model, model_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matriz = confusion_matrix(y_test, y_pred)
        exactitud = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        disp = ConfusionMatrixDisplay(matriz)
        model_preds.append([model_name, exactitud, precision, f1])
        #disp.plot()
        #print ("Confusion Matrix: ", matriz)
        #print ("Accuracy Score: ", exactitud)
        #print("Precision Score: ", precision)
        #print("F1 Score: ", f1)

    def model_eval():
        preds = pd.DataFrame(model_preds)
        preds.columns = ["Mod_Name", "Accuracy", "Precision", "F1 Score"]
        return preds.sort_values(by="Precision", ascending=False)
    
    print(f"DISPOSITIVO {i}")

    DTree_c = DecisionTreeClassifier(random_state=0)
    fit_model(DTree_c, "Dtree")

    #Rforest_c = RandomForestClassifier(random_state=0)
    #fit_model(Rforest_c, "Rforest")

    GBoost_c = GradientBoostingClassifier(random_state=0)
    fit_model(GBoost_c, "GBoost")

    logistic_regression_c = LogisticRegression(random_state=25)
    fit_model(logistic_regression_c, "Logistic Regression")

    print(model_eval())


In [19]:
for i in range(1):
    modelosPorDispositivo(i)

DISPOSITIVO 0
              Mod_Name  Accuracy  Precision  F1 Score
0                Dtree  0.788950   0.883727  0.878073
1               GBoost  0.870985   0.871027  0.931043
2  Logistic Regression  0.871011   0.871011  0.931059
DISPOSITIVO 1
              Mod_Name  Accuracy  Precision  F1 Score
0                Dtree  0.791037   0.886417  0.881186
2  Logistic Regression  0.884559   0.884559  0.938744
1               GBoost  0.884533   0.884556  0.938729
DISPOSITIVO 2
              Mod_Name  Accuracy  Precision  F1 Score
0                Dtree  0.642730   0.756732  0.749896
1               GBoost  0.720644   0.720738  0.837620
2  Logistic Regression  0.720695   0.720695  0.837679
DISPOSITIVO 3
              Mod_Name  Accuracy  Precision  F1 Score
0                Dtree  0.674514   0.763323  0.762217
1               GBoost  0.699189   0.707400  0.813461
2  Logistic Regression  0.685409   0.685409  0.813344
DISPOSITIVO 4
              Mod_Name  Accuracy  Precision  F1 Score
0           

In [37]:
from imblearn.over_sampling import RandomOverSampler

In [38]:
X = df1
y = df2.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=0) # 0.25 * 0.8 = 0.2
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [15]:
def objective_dtree(trial):
    max_depth = trial.suggest_int('max_depth', 2, 64)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
    
    clf = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=1)
    clf.fit(X_resampled,y_resampled)
    y_pred = clf.predict(X_test)
    f1=f1_score(y_test,y_pred,average='binary')
    
    return f1

sampler = optuna.samplers.TPESampler(seed=3535)
study_dtree = optuna.create_study(sampler=sampler, direction='maximize')
study_dtree.optimize(objective_dtree, n_trials=100)

[I 2024-04-14 05:18:25,414] A new study created in memory with name: no-name-8331ac01-eb35-4f13-b890-afd7312aa9e2
[I 2024-04-14 05:18:26,618] Trial 0 finished with value: 0.8161287195890277 and parameters: {'max_depth': 62, 'min_samples_leaf': 13}. Best is trial 0 with value: 0.8161287195890277.
[I 2024-04-14 05:18:27,687] Trial 1 finished with value: 0.7629880216783458 and parameters: {'max_depth': 24, 'min_samples_leaf': 12}. Best is trial 0 with value: 0.8161287195890277.
[I 2024-04-14 05:18:28,714] Trial 2 finished with value: 0.7964226565087779 and parameters: {'max_depth': 34, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.8161287195890277.
[I 2024-04-14 05:18:29,666] Trial 3 finished with value: 0.7882676136553988 and parameters: {'max_depth': 63, 'min_samples_leaf': 30}. Best is trial 0 with value: 0.8161287195890277.
[I 2024-04-14 05:18:30,582] Trial 4 finished with value: 0.75118736746175 and parameters: {'max_depth': 21, 'min_samples_leaf': 17}. Best is trial 0 with 

In [77]:
def objective_logistic(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2', 'l1'])
    
    clf = LogisticRegression(C=C, penalty=penalty, solver="liblinear",random_state=1)
    clf.fit(X_resampled, y_resampled)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='binary')
    
    return f1

sampler = optuna.samplers.TPESampler(seed=3535)
study_logistic = optuna.create_study(sampler=sampler, direction='maximize')
study_logistic.optimize(objective_logistic, n_trials=100)

C:\Users\Asus\AppData\Local\Temp\ipykernel_10616\3900807637.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
C:\Users\Asus\AppData\Local\Temp\ipykernel_10616\3900807637.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
C:\Users\Asus\AppData\Local\Temp\ipykernel_10616\3900807637.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
C:\Users\Asus\AppData\Local\Temp\ipykernel_106

In [69]:
print(study_logistic.best_params)

{'C': 1.0121374130447185e-05, 'penalty': 'l1'}


In [70]:
C=study_logistic.best_params['C']
penalty = study_logistic.best_params['penalty']

In [71]:
lr_clasification = LogisticRegression(C=C, penalty=penalty, solver="sag",random_state=1)

In [73]:
lr_clasification.fit(X_train,y_train)

#prediccion
y_pred = lr_clasification.predict(X_val)

#metricas
print(classification_report(y_val,y_pred))
print(f1_score(y_val,y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      5109
         1.0       0.87      1.00      0.93     33716

    accuracy                           0.87     38825
   macro avg       0.43      0.50      0.46     38825
weighted avg       0.75      0.87      0.81     38825

0.9295708633738161


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [33]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [35]:
def makePredictions(index):
    X = df1
    y = df2.iloc[:, index]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=0) # 0.25 * 0.8 = 0.2

    def objective_dtree(trial):
        max_depth = trial.suggest_int('max_depth', 2, 30)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
        
        clf = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=1)
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        f1=f1_score(y_test,y_pred,average='binary')
        
        return f1

    sampler = optuna.samplers.TPESampler(seed=3535)
    study_dtree = optuna.create_study(sampler=sampler, direction='maximize')
    study_dtree.optimize(objective_dtree, n_trials=100)

    print(study_dtree.best_params)
    #optuna.visualization.plot_optimization_history(study_dtree)

    max_depth=study_dtree.best_params['max_depth']
    min_samples_leaf = study_dtree.best_params['min_samples_leaf']


    #entrenamiento
    DTree_c = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf,random_state=0)
    DTree_c.fit(X_train,y_train)

    #prediccion
    y_pred = DTree_c.predict(X_val)

    #metricas
    print(classification_report(y_val,y_pred,))

    joblib.dump(DTree_c, f'modeloR_OP_D{index}.pkl')

In [36]:
for i in range(8):
    makePredictions(i)

{'max_depth': 6, 'min_samples_leaf': 13}
              precision    recall  f1-score   support

         0.0       0.33      0.00      0.00      5109
         1.0       0.87      1.00      0.93     33716

    accuracy                           0.87     38825
   macro avg       0.60      0.50      0.46     38825
weighted avg       0.80      0.87      0.81     38825



[W 2024-04-14 05:41:10,561] Trial 85 failed with parameters: {'max_depth': 23, 'min_samples_leaf': 28} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Asus\AppData\Local\Temp\ipykernel_10616\584211050.py", line 12, in objective_dtree
    clf.fit(X_train,y_train)
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py", line 959, in fit
    super()._fit(
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py", line 443, in _fit
    builder.build(self.tree_, X, y, sample_weight, missing_values_in_feature_

KeyboardInterrupt: 